In [1]:
#//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////// Libraries  ////////////////////////////////////////////////////////
import cip
from scapy.all import *
import pandas as pd
from decimal import Decimal
import time
import pandas as pd

In [2]:
current_results = "normal_behavior"
week = 0
analyzed_plc = "plc2"
path_in_plc_attack = current_results + "/week_" + str(week) + "/" 
path_out = current_results + "_" + analyzed_plc + "test_total_packets.csv"
network_data = rdpcap(path_in_plc_attack+analyzed_plc+"-eth0.pcap")
physical_data = pd.read_csv(path_in_plc_attack + "physical_process.csv")

In [3]:
tcp_packets_per_day = []
enip_packets_per_day = []
cip_packets_per_day = []
arp_packets_per_day = []
ts_phy_day_start = []

for i in range(1,7):
    # we subtract 28800, to have both times in GMT
    ts_phy_day_start.append((datetime.strptime(physical_data['Timestamps'].iloc[i*288], "%Y-%m-%d %H:%M:%S.%f") - datetime(1970, 1, 1)).total_seconds() - 28800 )

In [4]:
tcp_packet_count = 0
enip_packet_count = 0
cip_packet_count = 0
arp_packet_count = 0
current_day = 0
for pkt in network_data:
    # ARP packet
    if pkt['Ether'].type == 2054:
        arp_packet_count += 1
        continue
        
    if 'TCP' in pkt:
        if 'CIP' in pkt['TCP']:
            cip_packet_count+=1            
        elif 'ENIP_TCP' in pkt['TCP']:
            enip_packet_count+=1            
        else:
            tcp_packet_count += 1
           
    # Save this day's count, move one day, and restart the daily counters
    if float(pkt.time) >= ts_phy_day_start[current_day]:    
        tcp_packets_per_day.append(tcp_packet_count)
        enip_packets_per_day.append(enip_packet_count)
        cip_packets_per_day.append(cip_packet_count)
        arp_packets_per_day.append(arp_packet_count)    
        tcp_packet_count = 0
        enip_packet_count = 0
        cip_packet_count = 0
        arp_packet_count = 0        
        current_day += 1
        
        if current_day >= len(ts_phy_day_start):
            break          

In [5]:
print("TCP packets per day: " + str(tcp_packets_per_day))
print("ENIP packets per day: " + str(enip_packets_per_day))
print("CIP packets per day: " + str(cip_packets_per_day))
print("ARP packets per day: " + str(arp_packets_per_day))
total_packets = [tcp_packets_per_day, enip_packets_per_day, cip_packets_per_day, arp_packets_per_day]
days = ["TCP", "ENIP", "CIP", "ARP"]
numner_of_packets = pd.DataFrame(data = total_packets).T
numner_of_packets.columns = days
numner_of_packets.to_csv(path_out)

TCP packets per day: [16139, 16490, 16475, 16572, 16518, 16841]
ENIP packets per day: [4028, 4112, 4114, 4138, 4120, 4200]
CIP packets per day: [4028, 4112, 4114, 4137, 4121, 4200]
ARP packets per day: [2, 0, 0, 0, 0, 0]
